In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
#le module punkt fait référence à un modèle de tokenization de phrases appelé "Punkt Tokenizer". La tokenization de phrases est le processus de division d'un texte en phrases individuelles.
import nltk
import string 
from nltk.corpus import stopwords

from nltk.stem import LancasterStemmer
import re #EXPRESSION REGULIERE UTILISEE POUR MANIPULER DU TEXTE 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score,classification_report

In [2]:
file_path = "C:\\Users\\USER\\Downloads\\archive\\Genre Classification Dataset\\train_data.txt"
train_data = pd.read_csv(file_path,sep=':::',names=['Title','Genre','Description'])
file_path1 = "C:\\Users\\USER\\Downloads\\archive\\Genre Classification Dataset\\test_data.txt"
test_data = pd.read_csv(file_path1,sep=':::',names=['Title','Genre','Description'])

C:\Users\USER\AppData\Local\Temp\ipykernel_1236\805686882.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_data = pd.read_csv(file_path,sep=':::',names=['Title','Genre','Description'])
C:\Users\USER\AppData\Local\Temp\ipykernel_1236\805686882.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_data = pd.read_csv(file_path1,sep=':::',names=['Title','Genre','Description'])


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54214 entries, 1 to 54214
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        54214 non-null  object
 1   Genre        54214 non-null  object
 2   Description  54214 non-null  object
dtypes: object(3)
memory usage: 1.7+ MB


In [4]:
print(train_data.describe())

                                 Title    Genre  \
count                            54214    54214   
unique                           54214       27   
top      Oscar et la dame rose (2009)    drama    
freq                                 1    13613   

                                              Description  
count                                               54214  
unique                                              54086  
top      Grammy - music award of the American academy ...  
freq                                                   12  


In [5]:
print(train_data.isnull().sum())

Title          0
Genre          0
Description    0
dtype: int64


In [6]:
train_data.head()

,Title,Genre,Description
1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [7]:
#Tokenisation : Divisez le texte en mots ou en sous-parties significatives appelées tokens. 
#Vous pouvez utiliser des espaces 
#comme délimiteurs ou utiliser des outils plus avancés 
#comme la tokenisation basée sur des règles ou des bibliothèques de 
#traitement de langage naturel telles que NLTK (Natural Language Toolkit) ou spaCy.

# WE SHOULD CLEAN TEXT WE DEFINE A FUNCTION FOR THIS
import nltk
#nltk.download()

nltk.download()



showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [9]:
stemmer = LancasterStemmer()
stop_words = set(stopwords.words('english'))
def clean(dataText) :
    dataText = dataText.lower()
    dataText = re.sub(r'@\S+','',dataText)
    dataText = re.sub(r'http\S+','',dataText) # POUR SUPPRIMER LES LIENS 
    dataText = re.sub(r'pic.\S+','',dataText) # 
    dataText = re.sub(r"[^a-zA-Z+']", ' ', dataText)  # Keep only characters
    dataText = re.sub(r'\s+[a-zA-Z]\s+', ' ', dataText + ' ')  # Keep words with length > 1 only
    dataText = "".join([i for i in dataText if i not in string.punctuation])# supprimer la ponctuation 
    words = nltk.word_tokenize(dataText)
    stopwords = nltk.corpus.stopwords.words('english')  # Remove stopwords > les mot qui ont pas de sens comme MY ME THEIRE
    dataText = " ".join([i for i in words if i not in stopwords and len(i) > 2])
    dataText = re.sub("\s[\s]+", " ", dataText).strip()  # Remove repeated/leading/trailing spaces
    return dataText
#we should apply this function to the description column ;
train_data['clean_text'] = train_data['Description'].apply(clean)
test_data['clean_text'] = test_data['Description'].apply(clean)

In [10]:
train_data.head()

,Title,Genre,Description,clean_text
1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...,listening conversation doctor parents year old...
2,Cupid (1997),thriller,A brother and sister with a past incestuous r...,brother sister past incestuous relationship cu...
3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...,bus empties students field trip museum natural...
4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...,help unemployed father make ends meet edith tw...
5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...,films title refers recovered bodies ground zer...


In [11]:
test_data['clean_text']

0        brane loves life car apartment job especially ...
1        spain march quico naughty child three belongin...
2        one year life albin family shepherds north tra...
3        father died hasnt spoken brother years serious...
4        known internationally martial arts superstar b...
                               ...                        
54195    covering multiple genres tales light dark anth...
54196    alice cora munro attempt find father british o...
54197    movie years making oliver twist artful dodger ...
54198    popular mysterious rock mike mallard askew bro...
54199    curitiba city movement rhythms different pulsa...
Name: clean_text, Length: 54200, dtype: object

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [13]:
#Cet objet va être utilisé pour transformer le texte en vecteurs TF-IDF.
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(train_data['clean_text'])
X_test = tfidf_vectorizer.transform(test_data['clean_text'])

In [14]:
X = X_train
y = train_data['Genre'] # the target variable 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# cet objet permet de faire la classification 
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

In [16]:
# Make predictions on the validation set
y_pred = classifier.predict(X_val)

# Evaluate the performance of the model
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)
print(classification_report(y_val, y_pred, zero_division=0.0))

Validation Accuracy: 0.44526422576777647
               precision    recall  f1-score   support

      action        0.00      0.00      0.00       263
       adult        0.00      0.00      0.00       112
   adventure        0.00      0.00      0.00       139
   animation        0.00      0.00      0.00       104
   biography        0.00      0.00      0.00        61
      comedy        0.61      0.04      0.07      1443
       crime        0.00      0.00      0.00       107
 documentary        0.54      0.90      0.67      2659
       drama        0.38      0.88      0.53      2697
      family        0.00      0.00      0.00       150
     fantasy        0.00      0.00      0.00        74
   game-show        0.00      0.00      0.00        40
     history        0.00      0.00      0.00        45
      horror        0.00      0.00      0.00       431
       music        0.00      0.00      0.00       144
     musical        0.00      0.00      0.00        50
     mystery        0.0

In [17]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
# cet objet permet de faire la classification 
Logistic_classifier = LogisticRegression()
Logistic_classifier.fit(X_train, y_train)
# Make predictions on the validation set
y_pred = Logistic_classifier.predict(X_val)

# Evaluate the performance of the model
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)
print(classification_report(y_val, y_pred, zero_division=0.0))

C:\Users\USER\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Validation Accuracy: 0.5808355621138062
               precision    recall  f1-score   support

      action        0.56      0.22      0.32       263
       adult        0.82      0.21      0.33       112
   adventure        0.50      0.11      0.18       139
   animation        0.67      0.02      0.04       104
   biography        0.00      0.00      0.00        61
      comedy        0.53      0.59      0.56      1443
       crime        0.50      0.01      0.02       107
 documentary        0.65      0.86      0.74      2659
       drama        0.53      0.81      0.64      2697
      family        0.50      0.05      0.09       150
     fantasy        0.00      0.00      0.00        74
   game-show        0.93      0.35      0.51        40
     history        0.00      0.00      0.00        45
      horror        0.68      0.55      0.61       431
       music        0.69      0.38      0.49       144
     musical        0.00      0.00      0.00        50
     mystery        0.00

Alors le modele donne les meilleures performances!!

In [ ]:
svm_classifier = SVC(kernel="linear")
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_val)

# Evaluate the performance of the model
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)
print(classification_report(y_val, y_pred, zero_division=0.0))

In [19]:
import pickle
with open('modele_genre_films.pkl', 'wb') as f:
    pickle.dump(Logistic_classifier, f)

In [ ]:
X_test_predictions = classifier.predict(X_test)
test_data['Predicted_Genre'] = X_test_predictions

# Save the test_data DataFrame with predicted genres to a CSV file
test_data.to_csv('predicted_genres.csv', index=False)

# Display the 'test_data' DataFrame with predicted genres
print(test_data)